In [2]:
import os
os.chdir('../ppo_balancer/ppo_balancer/')

In [3]:
import argparse
import logging
from typing import Tuple
import time

import gin
import gymnasium as gym
import numpy as np
import upkie.envs
from envs import make_ppo_balancer_env
from settings import EnvSettings, PPOSettings, TrainingSettings
from stable_baselines3 import PPO
from upkie.utils.raspi import configure_agent_process, on_raspi
from upkie.utils.robot_state import RobotState
from upkie.utils.robot_state_randomization import RobotStateRandomization

upkie.envs.register()

/home/kevin/miniconda3/envs/ppo_balancer/lib/python3.12/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment UpkieGroundVelocity-v3 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/kevin/miniconda3/envs/ppo_balancer/lib/python3.12/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment UpkieServoPositions-v4 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/kevin/miniconda3/envs/ppo_balancer/lib/python3.12/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment UpkieServoTorques-v4 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/kevin/miniconda3/envs/ppo_balancer/lib/python3.12/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment UpkieServos-v4 already in registry.
  logger.warn(

In [4]:
def get_sagittal_force(intensity: float, duration=None):

    mass = 5.34  # mass in [kg]
    force = intensity*mass*9.81  # Force in [N]
    if duration is not None and duration > 1e9:
        force = 0.0
    return force

In [5]:
def run_policy(env: gym.Wrapper, policy) -> None:
    """!
    Run the policy on a given environment.

    @param env Upkie environment, wrapped by the agent.
    @param policy MLP policy to follow.
    """

    torso_force_in_world = np.zeros(3)
    bullet_action = {
        "external_forces": {
            "torso": {
                "force": torso_force_in_world,
                "local": False,
            }
        }
    }
    #Values of intensity sagitall force that is apply 
    intensity_force = 0.0
    isFall = False
    
    torso_force_in_world[0] = get_sagittal_force(intensity_force)
    
    best_magnitude = 0.0

    start = time.time_ns()

    action = np.zeros(env.action_space.shape)
    observation, info = env.reset()
    reward = 0.0
    while isFall==False:
        
        action, _ = policy.predict(observation, deterministic=True)
        now = time.time_ns()
        duration = now-start
        torso_force_in_world[0] = get_sagittal_force(intensity_force,duration = duration)

        env.bullet_extra(bullet_action)  # call before env.step

        observation, reward, terminated, truncated, info = env.step(action)
        if terminated or truncated or duration > 3e9:
            if terminated or truncated:
                print('est tombé')
                isFall = True
                best_magnitude = get_sagittal_force(intensity_force)
            start = time.time_ns()
            intensity_force += 0.01
            torso_force_in_world[0] = get_sagittal_force(intensity_force)
            observation, _ = env.reset()
    return best_magnitude

In [6]:
def main(policy_path: str, training: bool) -> None:
    """!
    Load environment and policy, and run the latter on the former.

    @param policy_path Path to policy parameters.
    @param training If True, add training noise and domain randomization.
    """
    env_settings = EnvSettings()
    init_state = None
    if training:
        training_settings = TrainingSettings()
        init_state = RobotState(
            randomization=RobotStateRandomization(
                **training_settings.init_rand
            ),
        )
    with gym.make(
        env_settings.env_id,
        frequency=env_settings.agent_frequency,
        init_state=init_state,
        max_ground_velocity=env_settings.max_ground_velocity,
        regulate_frequency=True,
        spine_config=env_settings.spine_config,
    ) as velocity_env:
        env = make_ppo_balancer_env(
            velocity_env,
            env_settings,
            training=training,
        )
        ppo_settings = PPOSettings()
        policy = PPO(
            "MlpPolicy",
            env,
            policy_kwargs={
                "net_arch": {
                    "pi": ppo_settings.net_arch_pi,
                    "vf": ppo_settings.net_arch_vf,
                },
            },
            verbose=0,
        )
        policy.set_parameters(policy_path)
        best_magnitude = run_policy(env, policy)
        print(f"MSFOS = {best_magnitude} N"  )

In [15]:
if on_raspi():
        configure_agent_process()

policy_path = f"policy/2024-12-04/mechanize_1/final.zip"
# Configuration
config_path = f"policy/operative_config.gin"
logging.info("Loading policy configuration from %s", config_path)
gin.parse_config_file(config_path)

try:
    main(policy_path, training=False)
except KeyboardInterrupt:
    logging.info("Caught a keyboard interrupt")


[2024-12-18 23:56:27,455] [info] Loading policy configuration from policy/operative_config.gin (2679653931.py:7)
[2024-12-18 23:56:27,488] [warning] UpkieGroundVelocity rate limiter is late by -1.500000 [ms] (rate_limiter.py:106)
[2024-12-18 23:56:30,494] [warning] UpkieGroundVelocity rate limiter is late by -1.500000 [ms] (rate_limiter.py:106)
[2024-12-18 23:56:33,502] [warning] UpkieGroundVelocity rate limiter is late by -1.500000 [ms] (rate_limiter.py:106)
[2024-12-18 23:56:36,509] [warning] UpkieGroundVelocity rate limiter is late by -1.500000 [ms] (rate_limiter.py:106)
[2024-12-18 23:56:39,516] [warning] UpkieGroundVelocity rate limiter is late by -1.600000 [ms] (rate_limiter.py:106)
[2024-12-18 23:56:42,523] [warning] UpkieGroundVelocity rate limiter is late by -1.500000 [ms] (rate_limiter.py:106)
[2024-12-18 23:56:45,530] [warning] UpkieGroundVelocity rate limiter is late by -1.500000 [ms] (rate_limiter.py:106)
[2024-12-18 23:56:48,537] [warning] UpkieGroundVelocity rate limiter

est tombé
MSFOS = 3.6669780000000003 N
